In [ ]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Relevancy_notebook",
        "spark.yarn.queue": "default",
        "spark.jars": "/apps/Jars/obelisk-retail-legos.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "2g",
        "spark.executor.memory": "2g",
        "spark.executor.instances": "4"
    }
}

In [ ]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

In [ ]:
import java.text.SimpleDateFormat
import java.time.DayOfWeek.{MONDAY, SUNDAY}
import java.time.format.DateTimeFormatter
import java.time.format.DateTimeFormatter.ofPattern
import java.time.temporal.TemporalAdjusters.previousOrSame
import java.time.temporal.{ChronoUnit, WeekFields}
import java.time.{LocalDate, LocalDateTime}
import java.util.{Date, Locale, TimeZone}  
def getFutureMonthDateFromHere(numMonths: Int, date: String, inputDateFormat: String,
                                 outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).plusMonths(numMonths).format(ofPattern(outputDateFormat))
  }

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession}
import java.time.format.DateTimeFormatter.ofPattern
import java.time.temporal.{ChronoUnit}
import java.time.{LocalDate}
import org.apache.spark.sql.types._

  val getNumOfMonthsBetweenTwoDates = udf((startDate: String, endDate: String)=> {

    val inputDateFormat: String = "yyyy-MM-dd"
    val dateBefore: LocalDate = LocalDate.parse(startDate, ofPattern(inputDateFormat))

    val dateAfter: LocalDate = LocalDate.parse(endDate, ofPattern(inputDateFormat))

    ChronoUnit.MONTHS.between(dateBefore, dateAfter)

  })

  val getOldMonthDateFromHere = udf((numMonths: Int, date: String) => {
      val inputDateFormat: String = "yyyy-MM-dd"
    val outputDateFormat: String = "yyyy-MM-dd"

    LocalDate.parse(date, ofPattern(inputDateFormat)).minusMonths(numMonths).format(ofPattern(outputDateFormat))
  })


    def getFutureMonthDateFromHere(numMonths: Int, date: String, inputDateFormat: String,
                                 outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).plusMonths(numMonths).format(ofPattern(outputDateFormat))
  }


  def getFutureDateFromHere(numDays: Int, date: String, inputDateFormat: String,
                            outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).plusDays(numDays).format(ofPattern(outputDateFormat))
  }

    def getOldDateFromHere(numDays: Int, date: String, inputDateFormat: String,
                         outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).minusDays(numDays).format(ofPattern(outputDateFormat))
  }

  def getOldMonthDateFromHerenotUdf(numMonths: Int, date: String, inputDateFormat: String,
                              outputDateFormat: String = "yyyy-MM-dd"): String = {
    LocalDate.parse(date, ofPattern(inputDateFormat)).minusMonths(numMonths).format(ofPattern(outputDateFormat))
  }

var DATE="date"
var PREDICTED="predicted"
var ACTUAL="actual"

var rundate = "2023-04-15"
var predictionend = getFutureMonthDateFromHere(1,rundate,"yyyy-MM-dd")
var dateForWhichPredictionsMade = getFutureDateFromHere(1,rundate,"yyyy-MM-dd")
var trainendDate =getOldDateFromHere(1,rundate,"yyyy-MM-dd")
var trainStartDate = "2022-01-15"

In [ ]:
def saveNLoadDF(key: String)(df: DataFrame): DataFrame = {
    var tempPath = "/data/Archive/bhavesh/inventoryPrediction/temp/dashboard_metrics"
    DFToParquet.putDF(tempPath + "/" + key, df)
    ParquetToDF.getDF(tempPath + "/" + key)
  }

def load(key: String): DataFrame = {
    var tempPath = "/data/Archive/bhavesh/inventoryPrediction/temp/dashboard_metrics"
    ParquetToDF.getDF(tempPath + "/" + key)
}

In [ ]:
var productLevelDailyQuantity = ParquetToDF.getDF("/data/Archive/inventory/2023-04-15/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/productlevelquantity")
productLevelDailyQuantity.printSchema

In [ ]:
var productLevelQuantity = (
    productLevelDailyQuantity.filter(col("date")>=trainStartDate && col("date")<dateForWhichPredictionsMade).filter(!(col("date")===rundate))
    .withColumn("pastmonth",getNumOfMonthsBetweenTwoDates(col(DATE),lit(trainendDate))+1)
    .filter(col("pastmonth") <= 15)
    .groupBy(PRODUCTID)
    .agg(sum(QUANTITY) as QUANTITY)
    .transform(saveNLoadDF("productQuantityForPast15MonthsAggregated"))
)

In [ ]:
var ga = (
    ParquetToDF.getDF("/data/ecomm/ajio/processed/ProcessedGAData")
    .select(PRODUCTID, "PLPViewsPerDay", DATE)
    .filter(col("date")>=trainStartDate && col("date")<dateForWhichPredictionsMade).filter(!(col("date")===rundate))
    .withColumn("pastmonth",getNumOfMonthsBetweenTwoDates(col(DATE),lit(trainendDate))+1)
    .filter(col("pastmonth") <= 15)
    .groupBy(PRODUCTID)
    .agg(sum("PLPViewsPerDay") as "impressions")
    .transform(saveNLoadDF("productImpressionsForPast15MonthsAggregated"))
)

In [ ]:
var productDetails = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/productDetails")
productDetails.printSchema

In [ ]:
productDetails = productDetails.select("productid").distinct().transform(saveNLoadDF("allProducts"))

In [ ]:
var productMetrics = productDetails.join(productLevelQuantity, Seq(PRODUCTID), "left").join(ga, Seq(PRODUCTID), "left").na.fill(0).transform(saveNLoadDF("productLevelMetrics"))

In [ ]:
var productMetrics = load("productLevelMetrics")

In [ ]:
var suffix = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"
var queriesToProductMap = ParquetToDF.getDF(s"/data/Archive/bhavesh/inventoryPrediction/queryToProductMap/date_when_prediction_is_made=$dateForWhichPredictionsMade/suffix=$suffix").select("query", "productid", "similargrouplevel", "template").na.drop()

In [ ]:
var queryLevelMetrics = (
    productMetrics.join(queriesToProductMap, Seq(PRODUCTID))
    .groupBy("query", "similargrouplevel", "template")
    .agg(sum(QUANTITY).cast(DoubleType) as QUANTITY, sum("impressions").cast(DoubleType) as "impressions")
    .transform(saveNLoadDF("queryLevelMetrics"))
)

In [ ]:
queryLevelMetrics.filter(col("query") === "4_full-length sleeve_black").show(false)

In [ ]:
778+480+465+531+804+718+654+477+1480+809+1065+1415+1258+1484+1223

In [ ]:
queryLevelMetrics.count

In [ ]:
var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions")
predictions.printSchema

In [ ]:
predictions.count

In [ ]:
var impressions = (
    predictions.select("query", "similargrouplevel", "predicted", "template")
    .join(queryLevelMetrics, Seq("query", SIMILAR_GROUP_LEVEL, "template"), "left")
    .withColumn("impressions", when(col("impressions") === 0 || col("quantity") === 0, lit(null)).otherwise((col("predicted") * col("impressions"))/col("quantity")))
    .select("query", "similargrouplevel", "impressions", "template")
)

DFToParquet.putDF("/data/Archive/bhavesh/inventoryPrediction/dashboardMetrics/impressions", impressions)
impressions = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/dashboardMetrics/impressions")

In [ ]:
impressions.filter(col("query") === "4_full-length sleeve_black").show(false)

In [ ]:
var impressions = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/dashboardMetrics/impressions")
impressions.printSchema

In [ ]:
var discountProductLevel = ParquetToDF.getDF("/data/Archive/inventory/2023-04-15/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/productDiscountDashboard")
discountProductLevel.printSchema

In [ ]:
var discountQueryLevel = (
    discountProductLevel.join(queriesToProductMap, Seq(PRODUCTID))
    .groupBy("query", "similargrouplevel", "template")
    .agg(avg("WeightedAvgDiscount").cast(DoubleType) as "discount")
    .transform(saveNLoadDF("queryLevelDiscount"))
)

In [ ]:
discountQueryLevel.filter(col("query") === "4_full-length sleeve_black").show(false)

In [ ]:
var discount = (
    predictions.select("query", "similargrouplevel", "template")
    .join(discountQueryLevel, Seq("query", SIMILAR_GROUP_LEVEL, "template"), "left")
    .na.fill(0.0)
    .select("query", "similargrouplevel", "discount", "template")
)

DFToParquet.putDF("/data/Archive/bhavesh/inventoryPrediction/dashboardMetrics/discount", discount)
discount = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction/dashboardMetrics/discount")

In [ ]:
discount.count

In [ ]:
impressions.count

In [ ]:
var dashboardMetrics = impressions.join(discount, Seq("query", "similargrouplevel", "template"))
println(dashboardMetrics.count)

In [ ]:
DFToParquet.putDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/dashboardMetrics", dashboardMetrics)
dashboardMetrics = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/dashboardMetrics")

In [ ]:
predictions.printSchema
dashboardMetrics.printSchema

In [ ]:
var predictionsWithDashboardMetrics = predictions.join(dashboardMetrics, Seq("query", "similargrouplevel", "template"))
predictionsWithDashboardMetrics.printSchema
println(predictionsWithDashboardMetrics.count)

In [ ]:
DFToParquet.putDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictionsWithDashboardMetrics", predictionsWithDashboardMetrics)